In [1]:
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  IS_IN_COLAB = True
  from google.colab import drive
  drive.mount('/content/gdrive')
else:
  print('Not running on CoLab')
  IS_IN_COLAB = False

Running on CoLab


Mounted at /content/gdrive


In [10]:
import os
os.getcwd()

'/content'

In [7]:
if 'google.colab' in str(get_ipython()):
  !ls -a /content/gdrive/MyDrive/

 Asset.gsheet		    "MLOPs Bay Area July '22 Recordings"
 Books			     Nguyen-Van-Thanh-KHHT.xlsx
'Colab Notebooks'	    'Personal Stuff'
 Datasets		    'Phan tich ky thuat.pptx'
'English Writing Practice'   PTKTCK.gdoc
 how_to_100m		    'Samsung Software Test'
 ml-25m.zip		    'Video Royal City'
 MLKubeflow.pdf		     Work


In [17]:
if IS_IN_COLAB:
  !cp /content/gdrive/MyDrive/ml-25m.zip .
  !unzip ml-25m.zip
  DATA_FOLDER = "."
else:
  DATA_FOLDER = "../../Datasets/"

Archive:  ml-25m.zip
   creating: ml-25m/
  inflating: ml-25m/tags.csv         
  inflating: ml-25m/links.csv        
  inflating: ml-25m/README.txt       
  inflating: ml-25m/ratings.csv      
  inflating: ml-25m/genome-tags.csv  
  inflating: ml-25m/genome-scores.csv  
  inflating: ml-25m/movies.csv       


In [14]:
import os, sys
from os.path import join, isdir, isfile
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import sys

In [15]:
class TRAIN_CONSTANTS:
    """PAD and MASK token to index
    """
    PAD = 0
    MASK = 1
    HISTORY = 120
    VOCAB_SIZE = 59049
    HEADS = 4
    LAYERS = 6
    EMB_DIM = 256
    MODEL_PATH = "../models/rec-transformer-model-2/model_files/bert4rec-state-dict.pth"

In [19]:
movies_df = pd.read_csv(join(DATA_FOLDER, 'ml-25m/movies.csv'))
print(len(movies_df))
print(movies_df.head())

62423
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


In [20]:
ratings_df = pd.read_csv(join(DATA_FOLDER, "ml-25m/ratings.csv"))
print(len(ratings_df))
print(ratings_df.head())

25000095
   userId  movieId  rating   timestamp
0       1      296     5.0  1147880044
1       1      306     3.5  1147868817
2       1      307     5.0  1147868828
3       1      665     5.0  1147878820
4       1      899     3.5  1147868510


In [22]:
ratings_df.sort_values(by=['timestamp'], inplace=True)
grouped_ratings = ratings_df.groupby(by='userId').agg(list)

In [23]:
print(grouped_ratings.head())

                                                  movieId  \
userId                                                      
1       [5952, 2012, 2011, 1653, 1250, 6539, 6377, 344...   
2       [2797, 5952, 1080, 553, 653, 497, 1374, 1653, ...   
3       [356, 593, 1270, 1, 480, 2571, 260, 318, 1196,...   
4       [97913, 93510, 91630, 93840, 195159, 122914, 1...   
5       [592, 590, 296, 150, 344, 153, 588, 595, 231, ...   

                                                   rating  \
userId                                                      
1       [4.0, 2.5, 2.5, 4.0, 4.0, 3.5, 4.0, 4.0, 4.0, ...   
2       [1.0, 5.0, 1.0, 2.0, 3.0, 4.0, 4.5, 4.5, 3.0, ...   
3       [4.0, 4.0, 3.5, 4.0, 2.0, 4.0, 4.0, 4.0, 4.0, ...   
4       [3.5, 4.0, 3.5, 4.5, 5.0, 3.0, 2.0, 3.5, 2.5, ...   
5       [3.0, 3.0, 4.0, 5.0, 4.0, 3.0, 4.0, 3.0, 4.0, ...   

                                                timestamp  
userId                                                     
1       [1147868053, 114

In [26]:
print(list(ratings_df.movieId.unique())[:10])

[21, 1079, 47, 1, 39, 32, 50, 52, 58, 2]


In [24]:
movieIdMapping = {k:i+2 for i, k in enumerate(sorted(list(ratings_df.movieId.unique())))}

In [25]:
print(len(movieIdMapping))

59047


In [29]:
# some basic info
print(max(movieIdMapping.keys()))
print(min(movieIdMapping.keys()))
print(max(movieIdMapping.values()))
print(min(movieIdMapping.values()))

209171
1
59048
2


In [30]:
ratings_df["movieId_mapped"] = ratings_df.movieId.map(movieIdMapping)

In [33]:
ratings_df.head()

,userId,movieId,rating,timestamp,movieId_mapped
326761,2262,21,3.0,789652009,22
326810,2262,1079,3.0,789652009,1054
326767,2262,47,5.0,789652009,48
15845015,102689,1,4.0,822873600,2
15845023,102689,39,5.0,822873600,40


In [34]:
movies_df["movieId_mapped"] = movies_df.movieId.map(movieIdMapping)

In [37]:
movies_df.to_csv(join(DATA_FOLDER, "ml-25m/movies_mapped.csv"), index = False)

In [38]:
ratings_df.to_csv(join(DATA_FOLDER, "ml-25m/ratings_mapped.csv"), index = False)

In [40]:
if IS_IN_COLAB:
  print("copying data")
  !cp ml-25m/ratings_mapped.csv /content/gdrive/MyDrive/
  !cp ml-25m/movies_mapped.csv /content/gdrive/MyDrive/

copying data
